In [1]:
import os
import warnings
warnings.filterwarnings("ignore")  
os.chdir("../../..")

In [60]:
import copy
from pathlib import Path
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [35]:
date=pd.date_range(start='2021/01/01',periods=1082,freq='d')
data=pd.DataFrame(np.random.randn(1082,3),columns=['A','B','C'])
data['date']=date
# add time index
data["time_idx"]=data['date'].dt.year*12+data['date'].dt.month
data["time_idx"]-=data["time_idx"].min()  #按年、月份设置索引
# data['date'].dt.weekofyear 输出当前日期为一年的第几周
# add additional features 
data["month"] = data.date.dt.month.astype(str).astype("category")   # categories have be strings
'''
transform:Call function producing a like-indexed Series on each group and return a Series having the same indexes as the original object.
例: grouped.transform(lambda x: (x - x.mean()) / x.std())
'''
data['A_mean']=data.groupby('month',observed=True).A.transform("mean")    #按month求A列的均值
    filled with the transformed values.
data

,A,B,C,date,time_idx,month,A_mean
0,-1.778283,-0.321081,0.837192,2021-01-01,0,1,0.192756
1,0.125001,-0.292223,0.650074,2021-01-02,0,1,0.192756
2,0.510933,-0.902860,-0.886443,2021-01-03,0,1,0.192756
3,1.019896,0.960692,0.064621,2021-01-04,0,1,0.192756
4,-0.845948,0.567268,1.821366,2021-01-05,0,1,0.192756
...,...,...,...,...,...,...,...
1077,0.203330,-0.489498,-0.129675,2023-12-14,35,12,0.111070
1078,0.415353,1.394505,-0.744979,2023-12-15,35,12,0.111070
1079,1.819536,0.385729,0.911125,2023-12-16,35,12,0.111070
1080,1.687120,1.225148,-0.154486,2023-12-17,35,12,0.111070


In [58]:
special_days = [
    "easter_day",
    "good_friday",
    "new_year",
    "christmas",
    "labor_day",
    "independence_day",
    "revolution_day_memorial",
    "regional_games",
    "fifa_u_17_world_cup",
    "football_gold_cup",
    "beer_capital",
    "music_fest",
]
data_b=pd.DataFrame([[1,1,1,1,1,0,0,0,0,0,0,1],[1,0,1,0,1,1,0,0,1,1,0,0]],columns=special_days)
data_b.apply(lambda x:x.map({0:"-",1:x.name})).astype('category')

,easter_day,good_friday,new_year,christmas,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest
0,easter_day,good_friday,new_year,christmas,labor_day,-,-,-,-,-,-,music_fest
1,easter_day,-,new_year,-,labor_day,independence_day,-,-,fifa_u_17_world_cup,football_gold_cup,-,-


In [56]:
TimeSeriesDataSet()

<bound method NDFrame._add_numeric_operations.<locals>.max of 0        1
1        1
2        1
3        1
4        1
        ..
1077    12
1078    12
1079    12
1080    12
1081    12
Name: month, Length: 1082, dtype: category
Categories (12, object): ['1', '10', '11', '12', ..., '6', '7', '8', '9']>